# 作业2 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [30]:
import pandas as pd

# 1. 从fundemantals.csv开始！

fundemantals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


## 回答

- **S&P500股票在2015年net income的均值是多少？**: 1560252997.752809
- **最大值比最小值多多少？**: 76922000000


- **S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？**: 24.43%
- **固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？**: LEN,TRV

In [31]:
df = pd.read_csv('fundamentals.csv')
df['date'] = pd.to_datetime(df['Period Ending']).map(lambda x: x.strftime('%Y'))
df.set_index('date')
data1 = df['Net Income'][df['date'] == '2015']

print(data1.mean())
print(data1.max() - data1.min())

1560252997.752809
76922000000.0


In [32]:
data2 = df[['Fixed Assets', 'Total Assets', 'Ticker Symbol']][df['date'] == '2016']
data2['percent'] = df['Fixed Assets'] / df['Total Assets'] * 100

print(data2.mean())
print(data2.min(axis=0))
data2.loc[data2['percent'] == 0]

Fixed Assets    7.228858e+09
Total Assets    2.971114e+10
percent         2.442821e+01
dtype: float64
Fixed Assets               0
Total Assets     1.77797e+09
Ticker Symbol            AAP
percent                    0
dtype: object


,Fixed Assets,Total Assets,Ticker Symbol,percent
950,0.0,1.536178e+10,LEN,0.0
1545,0.0,1.002450e+11,TRV,0.0


# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


## 回答

- 请列举出各个sector中的加入时间最早的股票名称： 答案见下表

In [33]:
df2 = pd.read_csv('securities.csv')
df2.dropna(subset=['Date first added'],inplace=True)

grouped = df2['Date first added'].groupby(df2['GICS Sector'])
d3 = grouped.min()
dict_d3 = {'GICS Sector':d3.index, 'Date first added':d3.values}
d3d = pd.DataFrame(dict_d3)
d4d = df2[['Security', 'GICS Sector', 'Date first added']]
d5d = pd.merge(d3d,d4d,how='inner')
d5d

,GICS Sector,Date first added,Security
0,Consumer Discretionary,1970-06-30,McDonald's Corp.
1,Consumer Staples,1969-03-31,General Mills
2,Consumer Staples,1969-03-31,The Clorox Company
3,Energy,1965-03-31,Schlumberger Ltd.
4,Financials,1975-06-30,JPMorgan Chase & Co.
5,Health Care,1964-03-31,Abbott Laboratories
6,Industrials,1964-03-31,Arconic Inc
7,Industrials,1964-03-31,Honeywell Int'l Inc.
8,Information Technology,1974-12-31,HP Inc.
9,Materials,1964-06-30,Sherwin-Williams


- 请列举出每一个州中加入时间最晚的股票名称： 答案见下表

In [34]:

grouped = df2['Date first added'].groupby(df2['Address of Headquarters'])
d3 = grouped.max()
dict_d3 = {'Address of Headquarters':d3.index, 'Date first added':d3.values}
d3d = pd.DataFrame(dict_d3)
d4d = df2[['Security', 'Address of Headquarters', 'Date first added']]
d5d = pd.merge(d3d,d4d,how='inner')
d5d

,Address of Headquarters,Date first added,Security
0,"Allentown, Pennsylvania",1985-04-30,Air Products & Chemicals Inc
1,"Amsterdam, New York",2013-12-23,Mohawk Industries
2,"Arlington, Virginia[3]",2007-01-10,"AvalonBay Communities, Inc."
3,"Atlanta, Georgia",2016-05-03,Acuity Brands Inc
4,"Auburn Hills, Michigan",2011-12-19,BorgWarner
...,...,...,...
182,"Winston-Salem, North Carolina",2015-03-20,Hanesbrands Inc
183,"Woburn, Massachusetts",2015-03-12,Skyworks Solutions
184,"Worsley, UK",2012-10-01,Pentair Ltd.
185,"York, Pennsylvania",2008-11-14,Dentsply Sirona


# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）:见下表
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）见下表

In [55]:

d6 = pd.merge(df, df2, left_on='Ticker Symbol', right_on='Ticker symbol')
grouped2 = d6['Research and Development'][d6['For Year'].between(2013,2016)].groupby(d6['GICS Sector'])
g2 = grouped2.sum()
g2

#每个sector在2013-2016年累计Research&Development的总投入

GICS Sector
Consumer Discretionary         1.860178e+09
Consumer Staples               8.557000e+08
Energy                         0.000000e+00
Financials                     0.000000e+00
Health Care                    6.334725e+10
Industrials                    4.219500e+09
Information Technology         1.986645e+11
Materials                      1.073121e+10
Real Estate                    0.000000e+00
Telecommunications Services    0.000000e+00
Utilities                      0.000000e+00
Name: Research and Development, dtype: float64

In [54]:
dict_d6d = {'GICS Sector': g2.index, 'sum':g2.values}
d6d = pd.DataFrame(dict_d6d)
d6d.sort_values('sum',ascending=False).head(n=3)

#下表为在2013-2016年累计Research&development投入最大的3家公司的名称

,GICS Sector,sum
6,Information Technology,1.986645e+11
4,Health Care,6.334725e+10
7,Materials,1.073121e+10


# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？